In [1]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import random

import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn import metrics
from scipy.optimize import curve_fit

import control
import make_query
import mining
import queries
import utils
import visualizations


### Importing Temporal User Data from:
- Uniswap
- Compound
- Decentraland
- AAVE
- Balancer
- 1INCH

The timeframe is from 1638334800 to 1648785600

In [2]:
# Open all the data

# Aggregate and add defi label

# bipatite matching like with amazon

In [3]:
aave = pd.read_csv("user_prediction_data/aave_data.csv")
axie = pd.read_csv("user_prediction_data/axie_data.csv")
balancer = pd.read_csv("user_prediction_data/balancer_data.csv")
compound = pd.read_csv("user_prediction_data/compound_data.csv")
decentraland = pd.read_csv("user_prediction_data/decentraland_data.csv")
oneInch = pd.read_csv("user_prediction_data/one_inch_data.csv")
uniswap = pd.read_csv("user_prediction_data/uniswap_data.csv")


In [4]:
def extract_users(df, col_name1, col_name2="", time_col="timestamp"):
    users = df[[col_name1, time_col]]
    users.columns = ["address", "time"]
    
    if col_name2 != "":
        more_users = df[[col_name2, time_col]]
        more_users.columns = ["address", "time"]
        users = pd.concat([users, more_users], axis=0)
    
    users.drop_duplicates()
    print(users.columns)
    
    return users


In [5]:
aave_users = extract_users(aave, "user.id")
aave_users["DeFi_APP"] = "Aave"
axie_users = extract_users(axie, "from.id", "to.id")
axie_users["DeFi_APP"] = "Axie"
balancer_users = extract_users(balancer, "userAddress.id")
balancer_users["DeFi_APP"] = "Balancer"
compound_users = extract_users(compound, "to", "from", "blockTime")
compound_users["DeFi_APP"] = "Compound"
decentraland_users = extract_users(decentraland, "buyer", "seller")
decentraland_users["DeFi_APP"] = "Decentraland"
one_inch_users = extract_users(oneInch, "from.id", "to.id")
one_inch_users["DeFi_APP"] = "1Inch"
uniswap_users = extract_users(uniswap, "sender", "to")
uniswap_users["DeFi_APP"] = "UniswapV2"


Index(['address', 'time'], dtype='object')
Index(['address', 'time'], dtype='object')
Index(['address', 'time'], dtype='object')
Index(['address', 'time'], dtype='object')
Index(['address', 'time'], dtype='object')
Index(['address', 'time'], dtype='object')
Index(['address', 'time'], dtype='object')


/var/folders/cs/cv1ys6cn52ld9z99yrbpcwbc0000gn/T/ipykernel_59676/2943698597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aave_users["DeFi_APP"] = "Aave"
/var/folders/cs/cv1ys6cn52ld9z99yrbpcwbc0000gn/T/ipykernel_59676/2943698597.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balancer_users["DeFi_APP"] = "Balancer"


In [6]:
dfs = [aave_users, axie_users, balancer_users, compound_users, decentraland_users, one_inch_users, uniswap_users]

user_data = pd.concat(dfs)

#user_data.drop_duplicates(subset=user_data.columns.difference(['time']))


In [7]:
sorted_user_data = user_data.sort_values(by = ["time"])
print(sorted_user_data.shape)

train_data = sorted_user_data.head(int(len(sorted_user_data)*(0.8)))
print(train_data.shape)


(301518, 3)
(241214, 3)


In [8]:
# this is generating a bipartite graph that is will have the the defi apps as one row and the addresses as another


def generate_bipartite_graph(df, source, target, edge):
    top = set(df[source])
    bot = set(df[target])
    
    G_nums = nx.from_pandas_edgelist(
        df, source, target, edge)
    edges = G_nums.edges(data=True)
    
    G_bi = nx.Graph()
    G_bi.add_nodes_from(top, bipartite=0)
    G_bi.add_nodes_from(bot, bipartite=1)
    G_bi.add_edges_from(edges)
    print(G_bi)
    
    return G_bi




In [9]:

# Dataframe to get counts for the number of times a user used a platform
df_count = sorted_user_data.groupby(
    ["address", "DeFi_APP"], as_index=False).size()


# testing on a random subset for visualization
df_count_small = df_count.sample(frac=0.2)
print(df_count_small.shape)

# generating the bipartite graph
G_bi = generate_bipartite_graph(df_count, "DeFi_APP", "address", "size")


(8945, 3)
Graph with 42266 nodes and 44724 edges


In [10]:
#nx.draw(G_bi)
nx.is_connected(G_bi)

True

In [11]:
# get layout
print("Getting Layout")
top = nx.bipartite.sets(G_bi)[0]
pos = nx.bipartite_layout(G_bi, top)

print("Getting adjacency matrix")
# get adjacency matrix
A = nx.adjacency_matrix(G_bi)
A = A.toarray()

print(A)



Getting Layout
Getting adjacency matrix
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [12]:
# Remove 25% of the edges
proportion_edges = 0.25
# this is our test set
edge_subset = random.sample(G_bi.edges(), int(proportion_edges * G_bi.number_of_edges()))

# Create a copy of the graph and remove the edges
B_train = G_bi.copy()
B_train.remove_edges_from(edge_subset)
print("train:", B_train)

B_test = nx.Graph()
B_test.add_edges_from(edge_subset)
print("train:", B_test)


/var/folders/cs/cv1ys6cn52ld9z99yrbpcwbc0000gn/T/ipykernel_59676/2499527120.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  edge_subset = random.sample(G_bi.edges(), int(proportion_edges * G_bi.number_of_edges()))


train: Graph with 42266 nodes and 33543 edges
train: Graph with 11013 nodes and 11181 edges


In [13]:
def MAP(G_test, G_pred, thres=0):
    # calculate avePrecision for each node and its neighbors
    avePs = []

    # loop through every node
    for node in tqdm(G_test.nodes()):
        # get predicted edges sorted in ranking order
        rankedPredWeights = sorted(
            G_pred[node].items(), key=lambda x: -x[1]['weight'])
        # only include edges that exist i.e. predicted rank / weight > threshold
        rankedPred = filter(
            lambda x: x[1]['weight'] > thres, rankedPredWeights)
        # get the rank
        pred = [x[0] for x in rankedPred]
        # calculate rel (existence of predicted edge in the groundtruth/actual set of edges)
        # get groundtruth neighbors
        gt = set(G_test[node])
        rel = np.array([x in gt for x in pred])
        # calculate P accumulative average of precision
        predLength = len(pred)
        P = np.array([
            sum(rel[:i+1])/len(rel[:i+1]) for i in range(predLength)
        ])
        # calculate aveP
        aveP = (rel @ P)/len(gt)
        # keep track of results
        avePs.append(aveP)
    MAPvalue = sum(avePs) / len(avePs)
    print("MAP: {}".format(MAPvalue))
    return MAPvalue


def ROC_PRC(pred, G):
    # prediction score
    y_score = [p[2] for p in pred]
    # groundtruth label
    y_true = [G.has_edge(p[0], p[1]) for p in pred]
    fig, (ax1, ax2) = plt.subplots(1, 2)
    # precision-recall curve
    fpr, tpr, thresholds = metrics.precision_recall_curve(y_true,  y_score)
    ax1.plot(fpr, tpr)
    ax1.set_title("Precision-Recall Curve")
    # receiver-operating characteristic curve
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)
    ax2.plot(fpr, tpr)
    ax2.set_title("ROC Curve, AUC = {:.2f}".format(
        metrics.roc_auc_score(y_true, y_score)))

    plt.show()


In [ ]:
for algo in [
    nx.resource_allocation_index,
    nx.jaccard_coefficient,
    nx.adamic_adar_index
]:
    print(algo)
    pred = list(algo(B_train))
    # create graph
    G_pred = nx.Graph()
    G_pred.add_weighted_edges_from(pred)
    # visualise adjacency matrix
    Apred = nx.adjacency_matrix(G_pred)
    Apred = Apred.toarray()
    #plt.imshow(Apred, cmap='Greys')
    #plt.show()
    # evaluation
    print("Visualizations for", algo)
    ROC_PRC(pred, G_bi)
    MAP(B_test, G_pred)


We see that these methods will only predict existing edges between the same sets. This is because two vertices from different sets will never share common neighbors, resulting in a 0 MAP score. 

In [14]:
A_train = nx.adjacency_matrix(B_train)
A_train = A_train.toarray()
A_train.shape




(42266, 42266)

In [15]:
# eigenvalue decomposition
V_train, U_train = np.linalg.eig(A_train)
print("Found Eigenvalue Decomposition!")
# U.T * Atest * U
target_V = U_train.T @ A @ U_train
print("Found Target!")
# take only the diagonals
target_V = np.diag(target_V)


In [120]:
# Looking at adding duplicates as teh weights --  imporvement is using freq and value transfered as weight

train_data_duplic_weights = train_data.groupby(["address", "DeFi_APP"], as_index=False).size()
train_data_duplic_weights = train_data_duplic_weights.merge(train_data, how="left")
train_data_duplic_weights


,address,DeFi_APP,size,time
0,0x0000000000005117dd3a72e64a705198753fdd54,Aave,6,1638566584
1,0x0000000000005117dd3a72e64a705198753fdd54,Aave,6,1638566584
2,0x0000000000005117dd3a72e64a705198753fdd54,Aave,6,1639735972
3,0x0000000000005117dd3a72e64a705198753fdd54,Aave,6,1639735972
4,0x0000000000005117dd3a72e64a705198753fdd54,Aave,6,1642881225
...,...,...,...,...
241209,0xffffa57756e1c19c1e0026487559982e721cffff,Compound,8,1640148472
241210,0xffffa57756e1c19c1e0026487559982e721cffff,Compound,8,1640148472
241211,0xffffa57756e1c19c1e0026487559982e721cffff,Compound,8,1640148472
241212,0xffffa57756e1c19c1e0026487559982e721cffff,Compound,8,1640148472


In [9]:
G = nx.from_pandas_edgelist(
    sorted_user_data, "address", "DeFi_APP", True)
print(G)


Graph with 42266 nodes and 44724 edges


In [122]:
edges = sorted(G.edges(data=True), key=lambda t: t[2].get('time', 1))


In [127]:
G_pred = nx.MultiGraph()


t_0 = G.size() / 4
counter = 0
max_k = 0

users = set()
apps = set()


In [74]:
for e in edges:

    G_pred.add_edge(e[0], e[1], attr=e[2])
    apps.add(e[0])
    users.add(e[1])

    
    if counter % 10000 == 0:
        print(("(Update)\t Counter", str(counter)))
        
    counter += 1
    if counter > t_0:

        dmax = sorted([d for n, d in G_pred.degree()], reverse=True)[0]

        User_preds = set()
        for v in users:
            for u in users:
                if u != v:

                    Nu = set(G_pred.neighbors(u))
                    Nv = set(G_pred.neighbors(v))   
                    print("u", u)
                    print("v", v)
                    print("Nu", Nu)
                    print("Nv", Nv)
                    print(G_pred.has_edge(v, u))
                    
                    if G_pred.has_edge(v, u) == False:

                        #print(len(Nv.intersection(Nu)), len(Nv.union(Nu)))
                        k = len(Nv.intersection(Nu)) / len(Nv.union(Nu))

                        if k > 0.5:
                            User_preds.add((v, u, k*5, 0))
            break
        break
    
print(G_pred)
print(len(users), len(apps))


NameError: name 'G_pred' is not defined

In [139]:
up_sample = list(User_preds)[0:100000]

In [165]:
import math
CN_preds = {}
AA_preds = {}
PA_preds = {}
TC_preds = {}
c = 0
for u in User_preds:

    frw = u[2]
    Nu = set(G_pred.neighbors(u[0]))
    Nfr = set(G_pred.neighbors(u[1]))
    print(Nu, Nfr, frw)
    
    
    N_inter = Nu.intersection(Nfr)

    Np = Nfr - Nu

    # ---------- Common Neighbors ----------------
    cn = len(N_inter)
    for prod in Np:
        if (u[0], prod) in CN_preds:
            CN_preds[(u[0], prod)] += cn * frw
        else:
            CN_preds[(u[0], prod)] = cn * frw
    break
    # ---------- Adamic-Adar ---------------------
    Au = 0
    for prod in N_inter:
        Au += 1/math.log(abs(len(set(G_pred.neighbors(prod)))))
        # Check that this is log10 or ln

    for prod in Np:
        if (u[0], prod) in AA_preds:
            AA_preds[(u[0], prod)] += Au * frw
        else:
            AA_preds[(u[0], prod)] = Au * frw

    # ---------- Peferential Attachment ----------
    PA = len(Nu) * len(Nfr)
    for prod in Np:
        if (u[0], prod) in PA_preds:
            PA_preds[(u[0], prod)] += PA * frw
        else:
            PA_preds[(u[0], prod)] = PA * frw
    
    # ---------- Strong Triadic Closure ----------
    diffs = 0.00001
    for prod in N_inter:
        e1 = G_pred.get_edge_data(u[0], prod)[0]["attr"]["size"]
        e2 = G_pred.get_edge_data(u[1], prod)[0]["attr"]["size"]

        diffs += abs(e2-e1)

    TC = diffs

    for prod in Np:
        if (u[0], prod) in TC_preds:
            TC_preds[(u[0], prod)] += TC
        else:
            TC_preds[(u[0], prod)] = TC
    if c% 1000000 == 0:
        print("(Update)\t Completed {0}/{1}".format(c, len(User_preds)))
    c+=1
    


{'Decentraland'} {'Decentraland'} 0


In [148]:
num_CN = 0
correct_CN = 0
for k, v in sorted(CN_preds.items(), key=lambda item: item[1], reverse=True):
    if num_CN < 100 and (G.has_edge(k[0], k[1])):
        correct_CN += 1
    num_CN += 1

num_AA = 0
correct_AA = 0
for k, v in sorted(AA_preds.items(), key=lambda item: item[1], reverse=True):
    if num_AA < 100 and (G.has_edge(k[0], k[1])):
        correct_AA += 1
    num_AA += 1

num_PA = 0
correct_PA = 0
for k, v in sorted(PA_preds.items(), key=lambda item: item[1], reverse=True):
    if num_PA < 100 and (G.has_edge(k[0], k[1])):
        correct_PA += 1
    num_PA += 1

num_TC = 0
correct_TC = 0
for k, v in sorted(TC_preds.items(), key=lambda item: item[1], reverse=False):
    if num_TC < 100 and (G.has_edge(k[0], k[1])):
        correct_TC += 1
    num_TC += 1


In [149]:
precision_common_neighbors = correct_CN/100
precision_adamic_adar = correct_AA/100
precision_pref_attachment = correct_PA/100
precision_triadic_closure = correct_TC/100
precision_personal_pagerank = 0.0

'''
Note: When i was experimenting with my results, I found that some of the
predicted edges with lower confidences were also correct, but were not 
picked up by the first 100.
'''

print("Precision common neighbors:", round(precision_common_neighbors, 4))
print("Precision Adamic/Adar:", round(precision_adamic_adar, 4))
print("Precision preferential attachment:",
      round(precision_pref_attachment, 4))
print("Precision triadic closure:", round(precision_triadic_closure, 4))



Precision common neighbors: 0.0
Precision Adamic/Adar: 0.0
Precision preferential attachment: 0.0
Precision triadic closure: 0.0


In [161]:
print(list(User_preds)[10000000:10000025])


[('0x4091243e3fb5e637d06c265c6eae1be7fb8460ce', '0x25fa459d179a73d66e977f22549563b92b3b9fbe', 5.0, 0), ('0x5eee6df063f6342994ccb8e5703f4e8961aa95cc', '0x74ec638d8e82ab4e7aacac77ef47a49e735ca7dd', 5.0, 0), ('0x5fcbbbaf2fe41036b4dc9f7b755746649f50c279', '0x5d25b9b5feafdce5f5fcc90b1f355249e50069e4', 5.0, 0), ('0xa3c04160d9ed4aefe11703e6584c43e97401beb2', '0x27ea64f3ea40a00e27198a248ed22b31579b3932', 5.0, 0), ('0x43b2cc9004630b2912abbce0fdf09ff5a3cc144b', '0xf06d597358fb28f86dd7eca8cad9a880cff60d2c', 5.0, 0), ('0xbd941a4b13e9343961dd135cf8226aedce565b72', '0x7b523a0c714988e5c4acaa71198ef0b52e0fb6b0', 5.0, 0), ('0x93ce08934372b78ba566151d0e02fa299a17ce80', '0xf667afda0c1cafcffca6ea6027b1c932f6dc7913', 5.0, 0), ('0xf749c623fabde451d480e79c6311ed6c62f2ab8c', '0xae6af1f93f7554e68ac48032eba670e1dc6b6c49', 5.0, 0), ('0x8962a5b11469f103f198978570024e18d15a1a30', '0xa9f35f607e2252084db230ff7ad50e57a946ab12', 5.0, 0), ('0x51607e7ad4d0cb771139c5190420387d67b1e1d6', '0x1f618d91dee238312552aeba562956c

pairs = [("0xe28e120834378746dd68a4f0b0ed6a4876352197", "0x75b0955d5727148816479881b7dd4cc646af27bb"), ("0xff618ea63335397a8acc2baf809701a865089489", "0x40486a01e98842821dd628084a99f67d9a80ebd2")]

In [162]:
sorted_user_data.loc[sorted_user_data["address"].isin(
    ["0x4091243e3fb5e637d06c265c6eae1be7fb8460ce", "0x25fa459d179a73d66e977f22549563b92b3b9fbe"])]

# This is missing a 1inch graph exchange row. 


,address,time,DeFi_APP
826,0x4091243e3fb5e637d06c265c6eae1be7fb8460ce,1582024460,Decentraland
14692,0x25fa459d179a73d66e977f22549563b92b3b9fbe,1625343152,Decentraland
24386,0x4091243e3fb5e637d06c265c6eae1be7fb8460ce,1642969311,Compound


In [18]:
Nu = list(set(G.neighbors("Decentraland")))
Nu[0]

'0x15fa2492b93a2f70772202835229f7ede0865a73'

44724